In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/gec-flan-t5-base/transformers/default/1/kaggle/working/gec_model1/config.json
/kaggle/input/gec-flan-t5-base/transformers/default/1/kaggle/working/gec_model1/spiece.model
/kaggle/input/gec-flan-t5-base/transformers/default/1/kaggle/working/gec_model1/training_args.bin
/kaggle/input/gec-flan-t5-base/transformers/default/1/kaggle/working/gec_model1/tokenizer_config.json
/kaggle/input/gec-flan-t5-base/transformers/default/1/kaggle/working/gec_model1/model.safetensors
/kaggle/input/gec-flan-t5-base/transformers/default/1/kaggle/working/gec_model1/special_tokens_map.json
/kaggle/input/gec-flan-t5-base/transformers/default/1/kaggle/working/gec_model1/added_tokens.json
/kaggle/input/gec-flan-t5-base/transformers/default/1/kaggle/working/gec_model1/generation_config.json
/kaggle/input/c4-000-010/C4_200M.tsv-00000-of-00010


In [3]:
!pip install -qU transformers datasets accelerate

In [ ]:
import pandas as pd

data = []
with open("/kaggle/input/c4-000-010/C4_200M.tsv-00000-of-00010", "r", encoding="utf-8") as f:
  for line in f:
    pieces = line.strip().split('\t')
    src = pieces[0]
    tgt = pieces[1]

    data.append([src, tgt])

df = pd.DataFrame(data, columns=["src", "tgt"])

In [ ]:
df.to_csv("data.csv", index=False)

In [4]:
from datasets import Dataset

dataset = Dataset.from_csv("/kaggle/working/data.csv")

In [5]:
dataset = dataset.shuffle().take(100000)

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

2025-05-19 20:17:00.715667: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747685820.740263    5921 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747685820.747865    5921 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface

In [7]:
def preprocess(example):
    input = ["gec: " + src for src in example["src"]]
    target = example["tgt"]
    result = tokenizer(input, text_target=target, truncation=True, padding=True)
    return result

In [8]:
tokenized_dataset= dataset.map(preprocess, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [9]:
tokenized_dataset = tokenized_dataset.remove_columns(['src','tgt'])

In [10]:
train_test_dataset = tokenized_dataset.train_test_split(test_size=0.02)

In [11]:
train_test_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 98000
})

In [12]:
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained("/kaggle/input/gec-flan-t5-base/transformers/default/1/kaggle/working/gec_model1").to(device)

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps = 1000,
    learning_rate=2e-5,
    bf16=True,
    num_train_epochs=1,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    save_total_limit=2,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_strategy="steps",
    save_steps = 1000,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=1000,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_test_dataset['train'],
    eval_dataset=train_test_dataset['test'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],

)

/tmp/ipykernel_5921/2017458482.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

In [15]:
trainer.save_model("gec_model_c4")

In [16]:
!zip -r gec_model_c4.zip /kaggle/working/gec_model_c4

  adding: kaggle/working/gec_model_c4/ (stored 0%)
  adding: kaggle/working/gec_model_c4/training_args.bin (deflated 51%)
  adding: kaggle/working/gec_model_c4/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/gec_model_c4/config.json (deflated 62%)
  adding: kaggle/working/gec_model_c4/added_tokens.json (deflated 83%)
  adding: kaggle/working/gec_model_c4/model.safetensors (deflated 7%)
  adding: kaggle/working/gec_model_c4/special_tokens_map.json (deflated 85%)
  adding: kaggle/working/gec_model_c4/generation_config.json (deflated 29%)
  adding: kaggle/working/gec_model_c4/spiece.model (deflated 48%)
